In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
from numba import njit, prange
import time
import pandas as pd

In [4]:
DIRECTIONS = np.array(
    [[1, 0], [-1, 0], [0, -1], [0, 1]], 
    dtype=np.float64
)

@njit
def visit(x,y, array, idx):
    "inputs pts where the walk visited"
    array[idx, 0] = x
    array[idx, 1] = y

@njit
def is_visited(x,y, array):
    "checks if the walk visited the pt (x,y)"
    for i in range(array.shape[0]):
        if array[i, 0] == x and array[i, 1] == y:
            return True
    return False

@njit
def random_walk2(steps):
    "creates a random walk of length steps"

    # a steps-length walk consists of steps+1 points
    array = np.zeros((steps+1, 2), dtype = np.float64)
    visit(0,0, array, 0)
    # current coordinates of walk
    x_coord, y_coord = 0, 0 
    q_x = 1
    
    for i in range(steps):
        neighbours = np.zeros((4,2), dtype = np.float64)
        count = 0

        for j in range(4):
            dx = DIRECTIONS[j, 0]
            dy = DIRECTIONS[j, 1]
            nx = x_coord + dx
            ny = y_coord + dy

            if not is_visited(nx, ny, array):
                # check if it is a possible neigbour
                neighbours[count, 0] = nx
                neighbours[count, 1] = ny
                count += 1

        possible_neighbours = neighbours[:count]

        if count == 0:
            # since it has no possible neighbour,
            # it is not a random walk of length steps (since it stays at the same spot)
            # so, p_x = 0
            # set q_x = 0 since it is also not a SAW of length steps (as it has no neighbours)
            # this avoids using the length of array to know if its a SAW or not
            return array, 0.0

        else:
            # walk moves to one of the possible neigbours randomly
            move = np.random.randint(count)
            x_coord = neighbours[move, 0]
            y_coord = neighbours[move, 1]
            visit(x_coord, y_coord, array, i+1)
            q_x = q_x * (1/ count)
            

    return array, q_x

In [6]:
@njit(parallel = True)
def estimation_saws(size, steps):
    "returns the estimated c_L value for L = steps"
    rus = np.zeros(size, dtype = np.float64)
    qxs = np.zeros(size, dtype = np.float64)

    for i in prange(size):
        _, q_x = random_walk2(steps)
        qxs[i] = q_x
        if q_x != 0:
            # rus[i] = ( (1/4)**steps) / q_x
            # but in c_L = (4**steps) * np.mean((qxs != 0) * rus)
            # when q_x = 0, we define ru = 0 to avoid dividing by 0
            # so if q_x is not a SAW of length steps, (q_x != 0) * ru = False (which is 0) * 0 = 0 
            # so for all the q_x not a SAW of length steps, np.mean( (qxs != 0) *rus) = np.mean(0) = 0
            # enough to consider np.mean(rus)
            # for c_L = (4**steps) * (np.mean( (qxs != 0) * rus)) = (4**steps) * (np.mean(rus))
            # since each ru = (1/4)**steps / q_x
            # c_L = np.mean(sum of all 1/q_x)\
            # do this to avoid floating-point precision error
            rus[i] = 1/q_x
    c_L = (np.mean(rus))
    # return c_L, count/len(qxs)
    return c_L

## Results for L = 1 to L = 15

In [7]:
L_vals = np.array([_ for _ in range(1, 16)])

In [8]:
start = time.time()
c_L_vals = np.array([estimation_saws(10**6, i) for i in L_vals])
mu_vals = c_L_vals ** (1/L_vals)
end = time.time()

print(f"Time taken: {end - start:.3f} seconds")

Time taken: 3.320 seconds


In [9]:
# c_L values for L = 1 to L = 15
c_L_vals

array([4.00000000e+00, 1.20000000e+01, 3.60000000e+01, 1.00003968e+02,
       2.83970772e+02, 7.80122664e+02, 2.17184166e+03, 5.91402708e+03,
       1.62686430e+04, 4.40765735e+04, 1.20217503e+05, 3.24993759e+05,
       8.81129435e+05, 2.37164581e+06, 6.41394715e+06])

In [10]:
# actual c_L vlaues for L = 1 to L = 15
act_cL = np.array([4,12,36,100,284,780,2172,5916,16268,44100,
 120292,324932,881500,2374444,6416596])

In [12]:
# relative error
r_err = np.abs( (c_L_vals - act_cL) / act_cL) * 100
r_err

array([0.        , 0.        , 0.        , 0.003968  , 0.01029155,
       0.01572615, 0.00729006, 0.03334888, 0.00395232, 0.05312139,
       0.06192985, 0.01900671, 0.04203805, 0.11784628, 0.04128124])